In [1]:
!pip install osmnx

import osmnx as ox
import heapq
import folium
from geopy.distance import geodesic
import networkx as nx
from google.colab import files  # Para fazer download do mapa gerado

# Função para calcular o caminho mais curto usando Dijkstra com min-heap
def dijkstra_min_heap(G, origem, destino):
    pq = [(0, origem)]  # (distância, nó)
    distancias = {v: float('inf') for v in G.nodes}
    distancias[origem] = 0
    predecessores = {v: None for v in G.nodes}

    while pq:
        distancia_atual, vertice_atual = heapq.heappop(pq)

        if vertice_atual == destino:
            break

        for vizinho, dados in G[vertice_atual].items():
            peso_aresta = dados.get('length', float('inf'))  # Acesso seguro ao 'length'
            nova_distancia = distancia_atual + peso_aresta

            if nova_distancia < distancias[vizinho]:
                distancias[vizinho] = nova_distancia
                predecessores[vizinho] = vertice_atual
                heapq.heappush(pq, (nova_distancia, vizinho))

    caminho = []
    atual = destino
    while atual is not None:
        caminho.append(atual)
        atual = predecessores[atual]
    caminho.reverse()

    return caminho, distancias[destino]

# Definir os 10 pares de pontos de interesse em Natal-RN
pares_pontos = [
    ("Praia de Ponta Negra", "Parque das Dunas"),
    ("Praia do Forte", "Aeroporto Internacional Aluízio Alves"),
    ("Centro Histórico de Natal", "Praia de Genipabu"),
    ("Museu Câmara Cascudo", "Shopping Midway Mall"),
    ("UFRN", "Praia de Cotovelo"),
    ("Teatro Alberto Maranhão", "Praia do Meio"),
    ("Natal Shopping", "Praia de Pirangi"),
    ("Arena das Dunas", "Praia de Pipa"),
    ("Praia de Areia Preta", "Museu de Cultura Popular Djalma Maranhão"),
    ("Mercado Municipal de Natal", "Shopping Via Direta")
]

# Carregar a rede viária de Natal
cidade = "Natal, Rio Grande do Norte, Brazil"
G = ox.graph_from_place(cidade, network_type='all')

# Calcular o comprimento das arestas
for u, v, data in G.edges(data=True):
    coords_u = (G.nodes[u]['y'], G.nodes[u]['x'])
    coords_v = (G.nodes[v]['y'], G.nodes[v]['x'])
    data['length'] = geodesic(coords_u, coords_v).meters  # Distância em metros

# Criar o mapa interativo com folium (apenas uma vez)
mapa = folium.Map(location=[-5.794, -35.211], zoom_start=12)  # Posição inicial centrada em Natal

# Variável para armazenar o último destino
ultimo_destino = None

# Para cada par de pontos, calcular o caminho mais curto e visualizar
for i, (origem, destino) in enumerate(pares_pontos):
    try:
        origem_coord = ox.geocode(origem + ", Natal, Rio Grande do Norte, Brazil")
        destino_coord = ox.geocode(destino + ", Natal, Rio Grande do Norte, Brazil")

        if origem_coord is None or destino_coord is None:
            print(f"Não foi possível encontrar um dos locais: {origem} ou {destino}")
            continue

        origem_nodo = ox.distance.nearest_nodes(G, origem_coord[1], origem_coord[0])
        destino_nodo = ox.distance.nearest_nodes(G, destino_coord[1], destino_coord[0])

        # **Cálculo do caminho com o Dijkstra com min-heap**:
        caminho_min_heap, distancia_min_heap = dijkstra_min_heap(G, origem_nodo, destino_nodo)
        print(f"Caminho (min-heap) de {origem} para {destino}: {caminho_min_heap}")
        print(f"Distância total (min-heap): {distancia_min_heap} metros")

        # **Cálculo do caminho com o Dijkstra do networkx**:
        caminho_networkx = nx.shortest_path(G, origem_nodo, destino_nodo, weight='length')
        distancia_networkx = nx.shortest_path_length(G, origem_nodo, destino_nodo, weight='length')
        print(f"Caminho (networkx) de {origem} para {destino}: {caminho_networkx}")
        print(f"Distância total (networkx): {distancia_networkx} metros")

        # Adicionar marcadores para os pontos de origem e destino
        folium.Marker([G.nodes[origem_nodo]['y'], G.nodes[origem_nodo]['x']], popup=f'{origem}').add_to(mapa)
        folium.Marker([G.nodes[destino_nodo]['y'], G.nodes[destino_nodo]['x']], popup=f'{destino}').add_to(mapa)

        # Adicionar a linha azul representando o caminho do Dijkstra (min-heap)
        for i in range(len(caminho_min_heap) - 1):
            u, v = caminho_min_heap[i], caminho_min_heap[i + 1]
            folium.PolyLine([(G.nodes[u]['y'], G.nodes[u]['x']), (G.nodes[v]['y'], G.nodes[v]['x'])], color='blue', weight=5, opacity=0.7).add_to(mapa)

        # Adicionar a linha vermelha representando o caminho do Dijkstra (networkx)
        for i in range(len(caminho_networkx) - 1):
            u, v = caminho_networkx[i], caminho_networkx[i + 1]
            folium.PolyLine([(G.nodes[u]['y'], G.nodes[u]['x']), (G.nodes[v]['y'], G.nodes[v]['x'])], color='red', weight=5, opacity=0.7).add_to(mapa)

        # Se for o primeiro par, não conecta ao último destino
        if ultimo_destino:
            # Conectar o último destino ao primeiro destino do próximo par
            folium.PolyLine([(G.nodes[ultimo_destino]['y'], G.nodes[ultimo_destino]['x']),
                             (G.nodes[origem_nodo]['y'], G.nodes[origem_nodo]['x'])], color='green', weight=5, opacity=0.7).add_to(mapa)

        # Atualizar o último destino para o destino atual
        ultimo_destino = destino_nodo

    except Exception as e:
        print(f"Erro ao processar o par {origem} -> {destino}: {e}")

# Salvar o mapa como um arquivo HTML
mapa.save("mapa_comparacao_dijkstra.html")

# Para download no Colab
files.download("mapa_comparacao_dijkstra.html")

# Ou para exibir diretamente no Colab
mapa


Caminho (min-heap) de Praia de Ponta Negra para Parque das Dunas: [1826735671]
Distância total (min-heap): inf metros
Caminho (networkx) de Praia de Ponta Negra para Parque das Dunas: [5062236379, 5062236377, 5062236384, 5062236382, 5062236397, 5062236395, 5062236405, 5062236409, 505036561, 3353160576, 505026778, 6135308937, 6135308932, 6135308926, 6135308920, 8648669938, 8648669939, 8636147723, 6427036363, 6427036364, 6427036362, 9592538958, 6459457350, 6427041195, 8648669937, 8648669936, 3670242027, 3377030458, 3377030453, 8636265557, 3377030448, 3377030445, 8633072093, 501925384, 6286855398, 501925831, 3377030431, 501925844, 501925847, 7629917324, 8633072092, 7629917322, 8633072097, 1944056919, 1944056913, 8633072100, 7158087673, 8629991599, 5157925563, 7629917326, 243207565, 300392684, 300392687, 243207566, 300392685, 6977211583, 8926993388, 7147885787, 503374076, 560213498, 503374098, 6991059596, 6991059599, 6991059140, 503374151, 8613147326, 8926993381, 301375993, 8926993372, 301

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>